# 10: Ingeniería de Características para Modelamiento

**Propósito:** Este *notebook* carga el `analytical_dataset.parquet` final y crea las *features* dinámicas y de comportamiento avanzadas (como se definen en el plan de investigación) necesarias para el modelamiento predictivo.

**Proceso:**
1.  **Cargar:** Cargar `analytical_dataset.parquet`.
2.  **Crear Features Fundacionales:**
    * Calcular el voto mayoritario de la bancada (Party Line) para cada votación.
    * Calcular la cohesión de la bancada (Índice de Rice) para cada votación.
    * Determinar si el legislador votó "con la bancada" (Booleano).
3.  **Crear Features de Comportamiento (Rolling):**
    * Calcular `disciplina_partidaria_historia` (ventana móvil).
    * Calcular `indice_discolo` (basado en Rice y voto).
4.  **Crear Features de Especialización:**
    * Calcular `entropia_tematica` (basado en *features* `ambito_`).
5.  **Crear Features de Distancia (Opcional - Matriz):**
    * Calcular `distancia_al_ejecutivo` y `centralidad_red`.
6.  **Guardar:** Guardar el *dataset* final listo para el modelo.

**Salidas (Artifacts):**
* `data/03_final/model_ready_dataset.parquet`

In [3]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
import sys
import json
from sklearn.preprocessing import MultiLabelBinarizer

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    # Necesitamos la función de normalizar texto para el merge de colegios
    from src.common_utils import normalize_string
except ImportError as e:
    logging.error(f"ERROR: No se pudo importar desde /src. {e}")
    raise

ImportError: cannot import name 'MultiLabelBinarizer' from 'sklearn.preprocessing' (unknown location)

In [4]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"
DATA_DIR_FINAL = ROOT / "data" / "03_final"

# (Directorio de donde cargas tu archivo de colegios)
DATA_DIR_RAW = ROOT / "data" / "01_raw" 

# Asegurarse que el directorio de salida exista
DATA_DIR_FINAL.mkdir(parents=True, exist_ok=True)

# --- Archivos de Entrada ---
DIPUTADOS_FILE = DATA_DIR_PROCESSED / "diputados_master_clean.parquet"
VOTACIONES_FILE = DATA_DIR_PROCESSED / "votaciones_master_clean.parquet"
BOLETINES_FILE = DATA_DIR_PROCESSED / "boletines_master_clean.parquet"
COLEGIOS_FILE = DATA_DIR_RAW / "colegios_chile.csv" 

logging.info(f"Directorio Procesado: {DATA_DIR_PROCESSED}")
logging.info(f"Directorio Final: {DATA_DIR_FINAL}")

2025-11-06 10:24:06,921 - INFO - Directorio Procesado: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\02_processed
2025-11-06 10:24:06,922 - INFO - Directorio Final: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\03_final


In [6]:
df_votaciones = pd.read_parquet(VOTACIONES_FILE)

AttributeError: module 'pandas' has no attribute 'read_parquet'

In [7]:
print(pd.__version__)
print(pd.__file__)

AttributeError: module 'pandas' has no attribute '__version__'

In [ ]:
!pip install --user --upgrade pandas